# Import uncertainty from file

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *
from bw_bamboo.uncertainty_importer import *

In [2]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - biogenic - air", 
        "CO2 - waste - fossil - air",
        "CO2 - agriculture - peat decay - air", 
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
FG_MAPPING_FILE = os.path.join(os.getcwd(), "data", "EXIOBASE-ecoinvent-bio-bw-GHG.csv")
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")
EMISSION_CODE_FILE = os.path.join(os.getcwd(), "data/EXIOBASE-ecoinvent-bio-bw-GHG.csv")
FROEGROUND_1 = os.path.join(os.getcwd(), "data/20250110_foreground_system_small.csv")
# FROEGROUND_1 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_1col.csv")
FROEGROUND_2 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_2col.csv")
mysector = 68  # TODO: why the first one is minus, 68 looks normal.

In [3]:
dp_builder = DatapackageBuilder()
uncertainty_handler = UncertaintyHandler()
uncetainty_importer = UncertaintyImporter(FROEGROUND_2, ",")

In [4]:
# import background system
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
bg_tech_matrix = bg_importer.form_tech_matrix(raw_tech)

bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bg_bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

cf_matrix = [1., 1., 1., 1., 1., 1., 27.0, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 273., 273., 25200.]
bg_cf_matrix = np.diagflat(cf_matrix)

In [5]:
# get activities
fg_activities = get_fg_activities(FROEGROUND_2, ",")
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
activities = fg_activities + bg_activities

In [6]:
# import foreground system
fg_importer = ForegroundImporter()
fg_tech_df = pd.read_table(FROEGROUND_2, sep=',')
fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_tech_df, GHG, fg_activities, bg_activities)
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(bg_tech_matrix, bg_bio_matrix, fgbg, fgfg, bgfg, bifg)

In [7]:
# import uncertainty
# uncetainty_importer.update_metadata_uncertainty(activities, "columnwise")
uncetainty_importer.update_metadata_uncertainty(activities, "itemwise")
import pprint
pprint.pprint(uncetainty_importer.metadata)

{0: {'Activity name': 'column_1',
     'Activity uncertainty type': 2,
     'Exchange negative': [False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           False,
                           True,
                           True,
                           False,
             

/Users/bp45th/Documents/github/brightway-exiobase/bw_bamboo/uncertainty_importer.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  negative_list = selected_df_2["Exchange negative"].replace(0, False).fillna(False).tolist()
/Users/bp45th/Documents/github/brightway-exiobase/bw_bamboo/uncertainty_importer.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  negative_list = selected_df_2["Exchange negative"].replace(0, False).fillna(False).tolist()


In [8]:
(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (bg_cf_data, bg_cf_indices) = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, bg_cf_matrix)

In [9]:
# full_tech_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(bg_tech_data, bg_tech_indices, bg_tech_flip, "columnwise")
# full_bio_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(bg_bio_data, bg_bio_indices, None, "columnwise")
full_tech_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_tech_data, full_tech_indices, full_tech_flip, "itemwise", fg_num=2, fg_strategy="itemwise")
full_bio_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_bio_data, full_bio_indices, None, "itemwise", fg_num=2, fg_strategy="itemwise")

# Here, we check the shape of the uncertainty array. As long as the data shape is identical to the uncertainty shape, it is correct.
# Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.
# print(full_tech_data.shape, full_bio_data.shape)
# print(full_tech_uncertainty_array.shape, full_bio_uncertainty_array.shape)
# print(full_tech_uncertainty_array)  # check the uncertainties are correct.

print(full_tech_uncertainty_array[:10])

uncertainty value:  0.1
uncertainty value:  0.2
uncertainty value:  2.674794737
uncertainty value:  2.674794737
uncertainty value:  2.2
uncertainty value:  2.674794737
uncertainty value:  2.674794737
uncertainty value:  2.3
uncertainty value:  2.674794737
uncertainty value:  2.288684673
uncertainty value:  2.4
uncertainty value:  2.5
uncertainty value:  2.6
uncertainty value:  2.238117654
uncertainty value:  2.699910682
uncertainty value:  1.1
uncertainty value:  1.56
uncertainty value:  2.9
uncertainty value:  0.0
uncertainty value:  0.0
[(2, 0.        , -2.3025851, nan, nan, nan, False)
 (2, 0.        , -1.609438 , nan, nan, nan, False)
 (0, 0.7363383 ,        nan, nan, nan, nan, False)
 (0, 0.00132513,        nan, nan, nan, nan, False)
 (0, 0.00250369,        nan, nan, nan, nan, False)
 (0, 0.00457712,        nan, nan, nan, nan, False)
 (0, 0.00210603,        nan, nan, nan, nan, False)
 (0, 0.00186496,        nan, nan, nan, nan, False)
 (0, 0.00194427,        nan, nan, nan, nan, Fal

In [10]:
dp_2 = dp_builder.prepare_datapackage([(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (bg_cf_data, bg_cf_indices)], [full_tech_uncertainty_array, full_bio_uncertainty_array, None])
dp_2.data

[array([( 0,  0), ( 1,  1), ( 2,  2), ..., (77, 75), (77, 76), (77, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([1.00000000e+00, 1.00000000e+00, 7.36338343e-01, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2, 0.0000000e+00, -2.3025851, nan, nan, nan, False),
        (2, 0.0000000e+00, -1.609438 , nan, nan, nan, False),
        (0, 7.3633832e-01,        nan, nan, nan, nan, False), ...,
        (0, 6.2417146e-03,        nan, nan, nan, nan, False),
        (0, 9.4099459e-04,        nan, nan, nan, nan, False),
        (0, 9.9517828e-01,        nan, nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False, False, False, ...,  True,  True, False]),
 array([(78,  2), (78,  3), (78,  4), ..., (97, 75), (97, 76), (97, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1.58717

In [11]:
lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[dp_2],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")
# static: Brightway calculated lca score: (53533247.217337795, 'RoW-Services')

InvalidParamsError: Real, positive scale (sigma) values are required for lognormal uncertainties.